**Import Modules**

In [ ]:
import numpy as np
import pandas as pd
from csv import reader
from matplotlib import pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]
plt.style.use("ggplot")

In [ ]:
def load_csv(filename):
    """
    Load data from a csv file 
    
    Parameters
    ----------
    filenanme: str
      Complete path of input dataset
    
    Returns
    -------
    List
      List of lists where each element represents a single record
    """
    dataset = list()
    with open(fileName, "r") as f:
        csv_reader = reader(f)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

**Import Raw Data as Pandas DataFrame**

In [ ]:
fileName = '../data/raw/ship_data.csv'
dataset = load_csv(fileName)
print("Loaded data with {0} rows and {1} columns".format(len(dataset), len(dataset[0])))

# Convert to pandas dataframe
colNames = dataset[0]
df = pd.DataFrame(dataset[1:], columns=colNames)

**Functions to convert unix timestamp to date-time object and re-index data frame, rename dataFrame columns to more readable names for easy referencing**

In [ ]:
def time_index(data, col_ts, ts_units):
    """
    Convert unix ts to date-time and re-index data-frame 
    
    Parameters
    ----------
    data: Pandas DataFrame
      Input pandas data frame object
    col_ts: str
      Column name with timestamp data
    ts_units: str
      Units of the timestamp. This could be 's' as seconds, refer to documentation for more options.
    
    Returns
    -------
    DataFrame
      Pandas DataFrame object with dateTime index
    """
    data[col_ts] = pd.to_datetime(data[col_ts], unit=ts_units)
    data.set_index(col_ts, inplace=True)
    return data

def rename_cols(data, new_col_names):
    """
    Rename columns 
    """
    data.columns = new_col_names
    return data    

In [ ]:
df = time_index(df, 'Time', 's')
new_col_names = ['fuelConsumption', 'HFO', 'MGO', 'draftForward', 'draftAft', 'draftMid1', 'draftMid2',
              'shaftSpeed', 'shaftTorque', 'shaftPower', 'speedGround', 'speedWater', 'heading', 'rudderAngle',
              'AWS', 'AWD', 'TWS', 'TWD', 'temp', 'currentDirection', 'currentSpeed', 'waterDepth', 'waveHeight',
              'wavePeriod', 'waveDirection']

df = rename_cols(df, new_col_names)
df.iloc[:6, :9]

**Summary Statistics**

In [ ]:
pd.set_option('display.width', 0)
pd.set_option('precision', 4)
description = dataset.describe()
description.iloc[:, :11]

**Feature correlations**

In [ ]:
pd.set_option('precision', 3)
cols_to_drop = ['HFO', 'MGO', 'currentDirection', 'currentSpeed', 'waterDepth', 'waveHeight', 'wavePeriod', 
                'waveDirection']
correlations = dataset.drop(cols_to_drop, axis=1).corr(method='pearson')
correlations.iloc[:10, :11]

**Correlation Heatmap**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,17,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(dataset.drop(cols_to_drop, axis=1).columns, rotation=90)
ax.set_yticklabels(dataset.drop(cols_to_drop, axis=1).columns);

In [ ]:
# Skewness of univariate distributions
dataset.skew()

In [ ]:
dataset.hist();

In [ ]:
dataset.plot(kind='density', subplots=True, layout=(5,5), sharex=False);

In [ ]:
dataset.plot(kind='box', subplots=True, layout=(5,5), sharex=False);

In [ ]:
# Missing values count plot
nan_all = dataset.isna().sum()
missing = nan_all[nan_all != 0].sort_values()

plt.style.use("ggplot")
fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(missing.index, missing.values, color='gray')
ax.set_xlabel("Fig1: Missing values count");

In [ ]:
# Zero values counts plot
zeroCounts_all = (dataset == 0).sum().sort_values()
zeroCounts = zeroCounts_all[zeroCounts_all > 500]

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(zeroCounts.index, zeroCounts.values, color='gray')
ax.set_xlabel("Fig2: Zero values count");

In [ ]:
df.dropna(how='any', inplace=True)

In [ ]:
df['meanDraft'] = df[['draftAft', 'draftForward', 'draftMid1', 'draftMid2']].mean(axis=1)

In [ ]:
# Drop the columns with high number of values zero - MGO, waveDirection, wavePeriod, waveHeight, waterDepth,
# currentDirection, currentSpeed, speedGround. Shaft power is output rather than input, shaft torque is not known
# before, so drop these too. AWS & AWD are dropped as their calculation involves speed of ship.
cols_to_drop = ['MGO', 'draftForward', 'draftAft', 'draftMid1', 'draftMid2', 'shaftTorque', 'shaftPower',
                'speedGround', 'AWS', 'AWD', 'currentDirection', 'currentSpeed', 'waterDepth',
                'waveHeight', 'wavePeriod', 'waveDirection']

df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df.loc[:, ['fuelConsumption', 'speedWater', 'shaftSpeed', 'temp']].plot(subplots=True, figsize=(15, 10))
plt.xlabel("Fig3: Year and month (dateTime)");

In [ ]:
df.loc[:, ['meanDraft', 'heading', 'rudderAngle', 'TWS']].plot(subplots=True, figsize=(15, 10))
plt.xlabel("Fig4: Year and month (dateTime)");

In [ ]:
# Summary Statistics - before data cleaning
df.describe().transpose()

In [ ]:
# Drop Absolute temp values (observed < -273), which are likely because of sensor error. Drop negative heading
# values and fuel consumption values < -0.5
conditionEval = (df['heading'] < 0) | (df['temp'] < -273) | (df['fuelConsumption'] < -0.5)
df.drop(df[conditionEval].index, inplace=True)
# Replace the Fuel consumption negative values between -0.5 and 0 with zero. These values are likely to be within
# standard error of measurement. Since shaft speed is zero for these values, it is inferred that the ship is
# docked at port.
df.loc[df['fuelConsumption'] < 0, 'fuelConsumption'] = 0

In [ ]:
# Summary Statistics - after data cleaning
df.describe().transpose()

In [ ]:
# Pair plot
sns.set_palette("RdBu_r")
g = sns.pairplot(df[df['fuelConsumption'] > 0], vars=['fuelConsumption', 'speedWater', 'shaftSpeed', 'temp'],
                 plot_kws={'alpha': 0.4})
g.fig.suptitle("Fig5: Univariate & bivariate distributions",  y=1.01);

In [ ]:
# Correlation Matrix
sns.heatmap(df.drop(columns='HFO').corr(), annot=True, cbar=False, linewidths=0.5);

In [ ]:
df.hist(color='grey');

In [ ]:
# Split-out validation dataset
dataArray = df.values
X = dataArray[:, 1:10]
y = dataArray[:, 0]
validation_size = 0.2
seed = 1
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, random_state=seed)

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 1
scoring = 'neg_mean_squared_error'

In [ ]:
# Spot-Check Algorithms
models = []
models.append(('LR', LinearRegression())) 
models.append(('LASSO', Lasso())) 
models.append(('EN', ElasticNet())) 
models.append(('KNN', KNeighborsRegressor())) 
models.append(('CART', DecisionTreeRegressor())) 
models.append(('SVR', SVR()))

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure() 
fig.suptitle('Algorithm Comparison') 
ax = fig.add_subplot(111) 
plt.boxplot(results, showmeans=True) 
ax.set_xticklabels(names);

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())]))) 
results = []
names = []
for name, model in pipelines:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Scaled Algorithm Comparison') 
ax = fig.add_subplot(111) 
plt.boxplot(results) 
ax.set_xticklabels(names);

**Ensembles**

In [ ]:
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor(n_estimators=10))]))) 
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor(n_estimators=10))])))
results = []
names = []
for name, model in ensembles:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Scaled Ensemble Algorithm Comparison') 
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names);

**Explore Number of trees and evaluate performance**

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    n_trees = [10, 50, 100, 300, 500, 1000, 1500, 2000]
    for n in n_trees:
        models[str(n)] = ExtraTreesRegressor(n_estimators=n)
    return models

models = get_models()

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

results = []
names = []
for name, model in models.items():
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, rescaledX, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True);

**Explore number of features**

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # Explore number of features to consider from 1 to 8
    for i in range(1, rescaledX.shape[1]):
        models[str(i)] = ExtraTreesRegressor(max_features=i)
    return models

models = get_models()

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

results = []
names = []
for name, model in models.items():
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, rescaledX, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True);

**Explore minimum samples per split**

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # Explore minimum samples per split from 2 to 8
    for i in range(2, 9):
        models[str(i)] = ExtraTreesRegressor(min_samples_split=i)
    return models

models = get_models()

In [ ]:
results = []
names = []
for name, model in models.items():
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, rescaledX, y_train, cv=kfold, scoring=scoring, n_jobs=14)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True);

**Final Model**

In [ ]:
# prepare the model
model = ExtraTreesRegressor(random_state=seed, 
                            n_estimators=100, 
                            max_features=5, 
                            min_samples_split=5, 
                            n_jobs=14)
model.fit(rescaledX, y_train)

In [ ]:
# transform the validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(y_validation, predictions))

In [ ]:
import seaborn as sns
sns.scatterplot(x = predictions, y = y_validation);